In [1]:
import matplotlib.pyplot as plt
from operator import itemgetter
import numpy as np
from math import *
import time
import os

In [2]:
def loadSingleAscDtm(path, sep):
    """
    """
    headerComponents = ['ncols', 'nrows', 'xllcenter', 'yllcenter', 'cellsize', 'nodata_value']
    
    with open(path, 'r') as f:
    
        counter = 0
        header = {}
        dtm = []

        for l in f:
            
            if l.endswith('\n'):
                l = l[:l.index('\n')]

            l = l.split(f'{sep}')

            if counter < 6:
                
                if any(map(lambda x: x in headerComponents, l)):
                    counter += 1
                    
                    try:
                        header[l[0]] = int(l[1])
                    
                    except:
                        header[l[0]] = float(l[1])

            else:
                heights = []
                
                for el in l:
                    
                    try:
                        heights.append(int(el))
                    
                    except:
                        heights.append(float(el))

                dtm.append(heights)
    
    return header, np.array(dtm)

In [3]:
def loadMultipleAscDtms(catalog, sep):
    """
    """
    headers = {}
    dtms = {}
    
    for file in os.listdir(catalog):
        
        if file.endswith('.asc'):
            header, dtm = loadSingleAscDtm(os.path.join(catalog, file), ' ')
            
            headers[file[:file.index('.')]] = header
            dtms[file[:file.index('.')]] = dtm
    
    return headers, dtms

In [4]:
def findStatisticsForDatasets(headers):
    """
    """  
    return {
        'mean cols num'  : int(np.mean([v['ncols'] for v in headers.values()])),
        'mean rows num'  : int(np.mean([v['nrows'] for v in headers.values()])),
        'mean cell size' : int(np.mean([v['cellsize'] for v in headers.values()])),
        'min X' : min([v['xllcenter'] for v in headers.values()]),
        'max X' : max([v['xllcenter'] for v in headers.values()]),
        'min Y' : min([v['yllcenter'] for v in headers.values()]),
        'max Y' : max([v['yllcenter'] for v in headers.values()]),
        'no data' : int(np.mean([v['nodata_value'] for v in headers.values()]))
    }

In [5]:
def findSpatialDistributionOfDatasets(headers, statistics):
    """
    """
    numOfDatasetsAlongX = int(round((statistics['max X'] - statistics['min X']) / statistics['mean rows num'] * \
                                    statistics['mean cell size'], 0)) + 1
    numOfDatasetsAlongY = int(round((statistics['max Y'] - statistics['min Y']) / statistics['mean cols num'] * \
                                    statistics['mean cell size'], 0)) + 1
    return numOfDatasetsAlongX, numOfDatasetsAlongY

def sortDatasets(headers, numOfDatasetsAlongX, numOfDatasetsAlongY):
    """
    """    
    sortedByX = list(zip([k for k in headers.keys()], [v['xllcenter'] for v in headers.values()], 
                          [v['yllcenter'] for v in headers.values()]))
    sortedByX.sort(key = itemgetter(1), reverse = True)
    
    splittedBySimilarX = [ sortedByX[ i : i + numOfDatasetsAlongY ] for i in range( 0, len(sortedByX), numOfDatasetsAlongY )]
    
    sortedByXY = []
    
    for i in range(len(splittedBySimilarX)):
        splittedBySimilarX[i].sort(key = itemgetter(2), reverse = False)
        
        tmp = []
        
        for j in range(len(splittedBySimilarX[i])):
            tmp.append(splittedBySimilarX[i][j][0])
        sortedByXY.append(tmp)
    
    return np.array(sortedByXY)

In [39]:
def sortDatasets(headers, numOfDatasetsAlongX, numOfDatasetsAlongY):
    """
    """    
    sortedByY = list(zip([k for k in headers.keys()], [v['xllcenter'] for v in headers.values()], 
                          [v['yllcenter'] for v in headers.values()]))
    sortedByY.sort(key = itemgetter(2), reverse = True)
    
    splittedBySimilarY = [ sortedByY[ i : i + numOfDatasetsAlongX ] for i in range( 0, len(sortedByY), numOfDatasetsAlongX )]
    
    sortedByXY = []
    
    for i in range(len(splittedBySimilarY)):
        splittedBySimilarY[i].sort(key = itemgetter(1), reverse = False)
        
        tmp = []
        
        for j in range(len(splittedBySimilarY[i])):
            tmp.append(splittedBySimilarY[i][j][0])
        sortedByXY.append(tmp)
    
    return np.array(sortedByXY)

def createFinalArrayFilledWithNoDataValues(statistics, headers, sortedDatasets):
    """
    """
    return np.ones((int((statistics['max X'] - statistics['min X']) / statistics['mean cell size'] + \
                        headers[sortedDatasets[0][-1]]['nrows'] * statistics['mean cell size']) ,
                    int((statistics['max Y'] - statistics['min Y']) / statistics['mean cell size'] + \
                        headers[sortedDatasets[0][-1]]['ncols'] * statistics['mean cell size'])), dtype = int) * statistics['no data']

In [107]:
sortedDatasets

array([['3830_441509_N-34-113-D-d-3-1', '3830_441510_N-34-113-D-d-3-2',
        '3830_441513_N-34-113-D-d-4-1'],
       ['3830_441511_N-34-113-D-d-3-3', '3830_441512_N-34-113-D-d-3-4',
        '3830_441515_N-34-113-D-d-4-3']], dtype='<U28')

In [108]:
def createFinalArrayFilledWithNoDataValues(statistics, headers, sortedDatasets):
    """
    """
    return np.ones((int((statistics['max Y'] - statistics['min Y']) / statistics['mean cell size'] + \
                        headers[sortedDatasets[0][-1]]['nrows']),
                     int((statistics['max X'] - statistics['min X']) / statistics['mean cell size'] + \
                        headers[sortedDatasets[-1][-1]]['ncols'])), dtype = int) * statistics['no data']

In [44]:
headers[sortedDatasets[0][-1]]

{'ncols': 2157,
 'nrows': 2359,
 'xllcenter': 597089.0,
 'yllcenter': 536726.0,
 'cellsize': 1.0,
 'nodata_value': -9999}

def fillFinalDtmArrayWithData(sortedDatasets, headers, statistics, dtms, finalDtmArray):
    """
    """
    for i in range(len(sortedDatasets) - 1, -1, -1):
        datasetsOnSimilarX = sortedDatasets[i]

        for j in range(len(datasetsOnSimilarX)):
            curDataset = datasetsOnSimilarX[j]

            xCur, yCur = headers[curDataset]['xllcenter'], headers[curDataset]['yllcenter']

            xDiff, yDiff = xCur - statistics['min X'], yCur - statistics['min Y']

            cellsDiffRow = int(xDiff / statistics['mean cell size'])
            cellsDiffCol = int(yDiff / statistics['mean cell size'])

            curRowInFinalDtmArray = int(-xDiff - 1)

            for k in range(dtms[curDataset].shape[0] - 1, -1, -1):
                curDatasetRow = dtms[curDataset][k]

                curColumnInFinalDtmArray = int(cellsDiffCol)

                for l in range(len(curDatasetRow)): 

                    if finalDtmArray[curRowInFinalDtmArray][curColumnInFinalDtmArray] == statistics['no data']:
                        finalDtmArray[curRowInFinalDtmArray][curColumnInFinalDtmArray] = curDatasetRow[l]

                    curColumnInFinalDtmArray += 1

                curRowInFinalDtmArray -= 1

In [120]:
statistics

{'mean cols num': 2157,
 'mean rows num': 2358,
 'mean cell size': 1,
 'min X': 592868.0,
 'max X': 597135.0,
 'min Y': 534327.0,
 'max Y': 536726.0,
 'no data': -9999}

In [122]:
def fillFinalDtmArrayWithData(sortedDatasets, headers, statistics, dtms, finalDtmArray):
    """
    """
    for i in range(len(sortedDatasets) - 1, -1, -1):
        datasetsOnSimilarY = sortedDatasets[i]

        for j in range(len(datasetsOnSimilarY)):
            curDataset = datasetsOnSimilarY[j]

            xCur, yCur = headers[curDataset]['xllcenter'], headers[curDataset]['yllcenter']

            xDiff, yDiff = xCur - statistics['min X'], yCur - statistics['min Y']

            xDiffAsCells = int(xDiff / statistics['mean cell size'])
            yDiffAsCells = int(yDiff / statistics['mean cell size'])          
            
            curDtm = dtms[curDataset]
            
            curRowInFinalDtmArray = -yDiffAsCells - 1
            
            for k in range(curDtm.shape[0] -1, -1, -1):
                curRowInCurDataset = curDtm[k]
                
                curColInFinalDtmArray = xDiffAsCells
                
                for l in range(len(curRowInCurDataset)):
                    curHeightInCurDataset = curRowInCurDataset[l]
                    
                    if finalDtmArray[curRowInFinalDtmArray][curColInFinalDtmArray] == statistics['no data']:
                        finalDtmArray[curRowInFinalDtmArray][curColInFinalDtmArray] = curHeightInCurDataset
                    
                    curColInFinalDtmArray += 1
                
                curRowInFinalDtmArray -= 1

In [123]:
def constructFinalHeader(finalDtmArray, statistics):
    """
    """
    return  {
            'ncols'        : finalDtmArray.shape[1],
            'nrows'        : finalDtmArray.shape[0],
            'xllcenter'    : statistics['min X'],
            'yllcenter'    : statistics['min Y'],
            'cellsize'     : statistics['mean cell size'],
            'nodata_value' : statistics['no data']
            }

In [124]:
def exportFinalDtmAsAscFile(catalog, finalHeader, finalDtmArray, sep):
    """
    """
    headerComponentsAndValues = list(zip(finalHeader.keys(), finalHeader.values()))
    headerComponentsAndValues = [f'{sep}'.join(str(el) for el in headerComponentsAndValues[i]) + '\n' for i \
                                 in range(len(headerComponentsAndValues))]
    
    dtmHeights = [ f'{sep}'.join(str(el) for el in finalDtmArray[i]) + '\n' for i in range(finalDtmArray.shape[0]) ]
    
    with open(os.path.join(catalog, 'merged_dtm.asc'), 'w') as f:
        for component in headerComponentsAndValues:
            f.write(component)
        for height in dtmHeights:
            f.write(height)

In [125]:
catalog = r'D:\WAT\DANE\INNE\08_2022\merge dtms'
sep = ' '

In [126]:
start = time.time()
headers, dtms = loadMultipleAscDtms(catalog, sep)
print(f'Data loading - execution time: {round(time.time() - start, 1)} [s]')

Data loading - execution time: 40.8 [s]


In [127]:
statistics = findStatisticsForDatasets(headers)

In [128]:
numOfDatasetsAlongX, numOfDatasetsAlongY = findSpatialDistributionOfDatasets(headers, statistics)

In [129]:
sortedDatasets = sortDatasets(headers, numOfDatasetsAlongX, numOfDatasetsAlongY)

C:\Users\pwroblewski\AppData\Local\Temp\ipykernel_15760\2747038239.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(sortedByXY)


In [130]:
finalDtmArray = createFinalArrayFilledWithNoDataValues(statistics, headers, sortedDatasets)

In [131]:
finalDtmArray.shape

(4758, 10693)

In [132]:
start = time.time()
fillFinalDtmArrayWithData(sortedDatasets, headers, statistics, dtms, finalDtmArray)
print(f'Merging DTMs - execution time: {round(time.time() - start, 1)} [s]')

0
0
44
0
2155
41
4267
83
0
2317
2110
2357
4221
2399
Merging DTMs - execution time: 27.0 [s]


In [133]:
finalHeader = constructFinalHeader(finalDtmArray, statistics)

{'ncols': 10693,
 'nrows': 4758,
 'xllcenter': 592868.0,
 'yllcenter': 534327.0,
 'cellsize': 1,
 'nodata_value': -9999}

In [134]:
exportFinalDtmAsAscFile(catalog, finalHeader, finalDtmArray, sep)